In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

from deep_fib.sci_net import SCIBlockCfg, SCINet
from deep_fib.data import DeepFIBDataset
from deep_fib.core import step_function

from utils.data import Marconi100Dataset, get_train_test_split
from utils.training import training_loop
from utils.summary import SummaryWriter

c:\Users\zumo0\anaconda3\envs\masterAI\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] Impossibile trovare la procedura specificata
  warn(f"Failed to load image Python extension: {e}")


In [3]:
train, test = get_train_test_split(0.5, 42)

TODO: salvare il dataset processato come nuovo dataset:
```
dataset
    |->unfolded
    |   |->r205n12_0.gzip
    |   |->r205n12_1.gzip
    |   |->...
    |   |->r256n18_n.gzip
    |->masks
    |   |->r205n12_mask_0.gzip
    |   |->r205n12_mask_1.gzip
    |   |->...
    |   |->r256n18_mask_n.gzip
    |->labels
        |->r205n12_label_0.gzip
        |->r205n12_label_1.gzip
        |->...
        |->r256n18_label_n.gzip
```

In [4]:
len(train), len(test)

(124, 125)

In [5]:
train = train[:10]
test = test[:4]
len(train), len(test)

(10, 4)

In [6]:
horizon = 1024
n_sample_per_data = 5

In [7]:
dataset_train = DeepFIBDataset(Marconi100Dataset(train, normalize=True), horizon, n_sample_per_data)
dataset_test = DeepFIBDataset(Marconi100Dataset(test, normalize=True), horizon, n_sample_per_data)

Unfolding: 100%|██████████| 4/4 [00:00<00:00, 30.41it/s]


In [8]:
data = dataset_train[0]
for d in data:
    print(d.size())

torch.Size([1024, 460])
torch.Size([1024, 460])
torch.Size([1024])


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_encoder_levels=2
input_dim=460

model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=None,
    block_config=SCIBlockCfg(
        input_dim=input_dim,
        hidden_size=2,
        kernel_size=3,
        dropout=0.5,
    )
).float().to(device)

In [10]:
batch_size = 2
num_workers = 0

train_loader = DataLoader(dataset_train, batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size, shuffle=False, num_workers=num_workers)

In [11]:
lr = 1e-3
step_size = 3

optim = Adam(model.parameters(), lr=lr)
lr_sched = StepLR(optim, step_size)

In [13]:
log_dir = "trash"

# torch.autograd.set_detect_anomaly(True)

with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        step_function=step_function,
        num_epochs=3*step_size,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir
    )

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [2, 460, 512]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
writer.launch()